In [ ]:
# importing libraries
from datetime import datetime, timezone, timedelta, time
import pandas as pd
import numpy as np
import talib
import csv
import time as tm
import re
import json
from tqdm import tqdm
import logging
import timeit
import concurrent.futures


# Spot Data

---

In [ ]:
# Reading Spot Data
df = pd.read_csv('data/2023/NiftySpotData-withSignal-with-closestExpiry-15min-2023.csv')

In [ ]:
print(df.tail(10))

In [ ]:
# Converting 1 min candle to 15 min candle

# Convert 'Datetime' column to datetime type if it's not already
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%Y-%m-%d %H:%M:%S%z')

# Determine the start time of the first interval
start_time_first_interval = df['Datetime'].iloc[0].replace(hour=9, minute=30, second=0, microsecond=0)

# Calculate the number of 15-minute intervals from the market opening time (9:30 AM)
df['Interval'] = ((df['Datetime'] - start_time_first_interval) // timedelta(minutes=15)).astype(int)

# Group by the new 'Interval' column and aggregate open, high, low, close values
result_df = df.groupby('Interval').agg({
    'High': 'max',
    'Low': 'min',
    'Open': 'first',
    'Close': 'last'
})

# Reset the index
result_df.reset_index(inplace=True)

# Calculate the datetime using the start time of the first interval
result_df['Datetime'] = start_time_first_interval + result_df['Interval'] * timedelta(minutes=15)

# Drop the 'Interval' column if you don't need it in the final result
result_df.drop(columns='Interval', inplace=True)

result_df['Datetime'] = pd.to_datetime(result_df['Datetime'], format='%Y-%m-%d %H:%M:%S%z')

df = result_df


In [ ]:
# Calculating the ADM (adapting moving average)
def adaptive_moving_average(close_prices, window=14, fast_factor=2.0, slow_factor=30.0):
    volatility = close_prices.pct_change().rolling(window=window, min_periods=window).std()
    fast_ema = close_prices.ewm(span=fast_factor * window, adjust=False).mean()
    slow_ema = close_prices.ewm(span=slow_factor * window, adjust=False).mean()
    ama = (fast_ema + volatility * (close_prices - slow_ema)).ewm(span=window, adjust=False).mean()
    return ama

df['ama'] = adaptive_moving_average(df['Close'], window=14, fast_factor=2, slow_factor=30)

In [ ]:
# Calculating the RSI
df['rsi'] = talib.RSI(df['Close'], timeperiod=14)

In [ ]:
# Calculate ATR
pd.set_option("display.max_rows", None, "display.max_columns", None)
np.set_printoptions(suppress=True,linewidth=np.nan)
pd.options.display.float_format = '{:,.4f}'.format


def calculate_atr(data, period=14):
    # Calculate True Range (TR)
    data['high-low'] = data['High'] - data['Low']
    data['high-close_prev'] = abs(data['High'] - data['Close'].shift(1))
    data['low-close_prev'] = abs(data['Low'] - data['Close'].shift(1))
    data['true_range'] = data[['high-low', 'high-close_prev', 'low-close_prev']].max(axis=1)

    # Calculate ATR
    data['atr'] = data['true_range'].rolling(window=period, min_periods=1).mean()

    # Drop intermediate columns
    data.drop(['high-low', 'high-close_prev', 'low-close_prev', 'true_range'], axis=1, inplace=True)

    return data


df = calculate_atr(df, period=14)


In [ ]:
# Calculating Pivot Points
def calculate_pivot_points(data, column_name, length=14):
    if column_name == 'High':
        pivot_points = pd.DataFrame(index=data.index, columns=[column_name, f'IsPivot{column_name.capitalize()}'])

        for i in range(length, len(data) - length):
            # Check if the current point is a pivot high
            is_pivot_point = all(data[column_name][i] > data[column_name][i - length:i]) and all(data[column_name][i] > data[column_name][i + 1:i + length + 1])
            # is_pivot_point = all(data[column_name][i] > data[column_name][i - length:i]) 

            # Store the values in the DataFrame
            pivot_points.at[data.index[i], column_name] = data[column_name][i] if is_pivot_point else None

            pivot_points.at[data.index[i], f'IsPivot{column_name.capitalize()}'] = is_pivot_point
    
    elif column_name == 'Low':
        pivot_points = pd.DataFrame(index=data.index, columns=[column_name, f'IsPivot{column_name.capitalize()}'])
        for i in range(length, len(data) - length):
            # Check if the current point is a pivot low
            is_pivot_point = all(data[column_name][i] < data[column_name][i - length:i]) and all(data[column_name][i] < data[column_name][i + 1:i + length + 1])
            # is_pivot_point = all(data[column_name][i] < data[column_name][i - length:i])

            # Store the values in the DataFrame
            pivot_points.at[data.index[i], column_name] = data[column_name][i] if is_pivot_point else None

            pivot_points.at[data.index[i], f'IsPivot{column_name.capitalize()}'] = is_pivot_point

    return pivot_points


PH = calculate_pivot_points(df, 'High', length=14)
df['PL'] = calculate_pivot_points(df, 'Low', length=14)['IsPivotLow']

df['PH'] = PH['IsPivotHigh'] 


for i in range(1, len(df)):
    if df.at[df.index[i], 'PH']:
        df.at[df.index[i], 'PH_val'] = df.at[df.index[i], 'High']
    else:
        df.at[df.index[i], 'PH_val'] = df.at[df.index[i - 1], 'PH_val']
    if df.at[df.index[i], 'PL']:
        df.at[df.index[i], 'PL_val'] = df.at[df.index[i], 'Low']
    else:
        df.at[df.index[i], 'PL_val'] = df.at[df.index[i - 1], 'PL_val']

df['Slope'] = df['atr']/14

# Initialize the first values for slope_ph and slope_pl
df.at[df.index[0], 'slope_ph'] = df.at[df.index[0], 'Slope']
df.at[df.index[0], 'slope_pl'] = df.at[df.index[0], 'Slope']


# Update slope_ph and slope_pl based on conditions
for i in range(1, len(df)):
    if df.at[df.index[i], 'PH']:
        df.at[df.index[i], 'slope_ph'] = df.at[df.index[i], 'Slope']
    else:
        df.at[df.index[i], 'slope_ph'] = df.at[df.index[i - 1], 'slope_ph']
    if df.at[df.index[i], 'PL']:
        df.at[df.index[i], 'slope_pl'] = df.at[df.index[i], 'Slope']
    else:
        df.at[df.index[i], 'slope_pl'] = df.at[df.index[i - 1], 'slope_pl']



df['upper'] = 0
df['lower'] = 0
# Update upper and lower based on conditions

for i in range(1, len(df)):
    if df.at[df.index[i], 'PH'] == True:
        df.at[df.index[i], 'upper'] = df.at[df.index[i], 'High']
    else:
        df.at[df.index[i], 'upper'] = df.at[df.index[i - 1], 'upper'] - df.at[df.index[i], 'slope_ph']

for i in range(1, len(df)):
    if df.at[df.index[i], 'PL'] == True:
        df.at[df.index[i], 'lower'] = df.at[df.index[i], 'Low']
    else:
        df.at[df.index[i], 'lower'] = df.at[df.index[i - 1], 'lower'] + df.at[df.index[i], 'slope_pl']
        

# length = 14
df['upos'] = 0
df['dnos'] = 0

# Calculate upos
for i in range(1, len(df)):
    if df.at[df.index[i], 'PH'] != True or df.at[df.index[i], 'PL'] =='':
        upper_limit = df.at[df.index[i - 1], 'upper']
        if df.at[df.index[i], 'Close'] > upper_limit:
            df.at[df.index[i], 'upos'] = 1

# Calculate dnos
for i in range(1, len(df)):
    if df.at[df.index[i], 'PL'] != True or df.at[df.index[i], 'PH'] == '':
        lower_limit = df.at[df.index[i - 1], 'lower']
        if df.at[df.index[i], 'Close'] < lower_limit:
            df.at[df.index[i], 'dnos'] = 1



In [ ]:
# Calulating the pivot highs and pivot lows
pivot = calculate_pivot_points(df, 'High', length=14)
df['IsPivotHigh'] = pivot['IsPivotHigh']

In [ ]:
# Calculating the signals

df['signal'] = 'Hold'

buy_condition = (df['upos'] > df['upos'].shift(1)) & (df['signal'] != 'Buy')
sell_condition = (df['dnos'] > df['dnos'].shift(1)) & (df['signal'] != 'Sell')

df.loc[buy_condition, 'signal'] = 'Buy'
df.loc[sell_condition, 'signal'] = 'Sell'




In [ ]:
df.tail(10)

In [ ]:
# Saving the data SPOT to a csv file
df.to_csv('data/2023/NiftySpotData-withSignal-with-closestExpiry-15min-2023.csv', index=False)


Formating code

In [ ]:
# convert closest_expiry to %D%B%Y format in all upper case
df['dby'] = df['closest_expiry'].dt.strftime('%d%b%y').str.upper()

# keep values that are in between 9:15 to 15:15 including them
df = df[(df['Datetime'].dt.time >= time(9, 15)) & (df['Datetime'].dt.time <= time(15, 15))]


In [ ]:
df.to_csv('NiftySpotData1-1.csv')

# Options Data

---

In [1]:
# importing libraries
from datetime import datetime, timezone, timedelta, time
import pandas as pd
import numpy as np
import talib
import csv
import time as tm
import re
import json
from tqdm import tqdm
import logging
import timeit
import concurrent.futures


In [2]:
year = 2024

In [3]:
df = pd.read_csv(f'data/{year}/NiftySpotData-{year}-15min-Signal-Expiry.csv')

df['Datetime'] = pd.to_datetime(df['Datetime'], format='%Y-%m-%d %H:%M:%S%z')

if year == 2024:
    df['closest_expiry'] = pd.to_datetime(df['closest_expiry'], format='%Y-%m-%d')
    # convert to %Y-%m-%d %H:%M:%S%z format
    df['closest_expiry'] = df['closest_expiry'].dt.strftime('%Y-%m-%d %H:%M:%S%z')
else:
    df['closest_expiry'] = pd.to_datetime(df['closest_expiry'], format='%Y-%m-%d %H:%M:%S%z')

try:
    # df drop PL PH, PL_val, PH_val, slope_ph, slope_pl, upper, lower, upos, dnos, IsPivotHigh
    df.drop(['PL', 'PH', 'PL_val', 'PH_val', 'slope_ph', 'slope_pl', 'upper', 'lower', 'upos', 'dnos', 'IsPivotHigh', 'Slope'], axis=1, inplace=True)
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
except:
    pass

# Initialize all columns with NaN
df = df.assign(
    position='', atmSP=np.nan, wingCall=np.nan, wingPut=np.nan, legPriceOrignal1=np.nan, legPriceOrignal2=np.nan,
    legPriceOrignal3=np.nan, legPriceOrignal4=np.nan, lpos1=np.nan, lpos2=np.nan, lpos3=np.nan, lpos4=np.nan,
    legAfterPos1=np.nan, legAfterPos2=np.nan, legAfterPos3=np.nan, legAfterPos4=np.nan, legAfterPosDiff1=np.nan,
    legAfterPosDiff2=np.nan, legAfterPosDiff3=np.nan, legAfterPosDiff4=np.nan, legPriceFinal1=np.nan,
    legPriceFinal2=np.nan, legPriceFinal3=np.nan, legPriceFinal4=np.nan, m2m1=np.nan, m2m2=np.nan, m2m3=np.nan,
    m2m4=np.nan, totalPL=0, cumReturns=0, balance=0
)

# only keep values between 9:15 to 15:30
df = df[(df['Datetime'].dt.time >= time(9, 15)) & (df['Datetime'].dt.time <= time(15, 30))]

# reset df index
df.reset_index(drop=True, inplace=True)


FileNotFoundError: [Errno 2] No such file or directory: 'data/2024/NiftySpotData-2024-15min-Signal-Expiry.csv'

In [4]:
# account for missing optionsData by dropping the rows from df where Datetime is
# dates_to_drop = ['2023-02-16', '2023-03-07', '2023-03-30', '2023-04-04', '2023-09-19', '2023-04-25', '2023-01-20', '2023-10-22', '2023-10-24']
# dates_to_drop = ['2022-09-22']
# dates_to_drop = ['2021-01-08', '2021-02-24', '2021-07-16', '2021-08-02', '2021-11-03', '2021-11-04']
# dates_to_drop = ['2020-01-31', '2020-03-13']
dates_to_drop = ['2024-03-02']
for date in dates_to_drop:
    df = df[df['Datetime'].dt.date != pd.to_datetime(date).date()]

df.reset_index(drop=True, inplace=True)

NameError: name 'dates_to_drop' is not defined

In [5]:
df.head()

,High,Low,Open,Close,Datetime,ama,rsi,atr,signal,closest_expiry,...,legPriceFinal2,legPriceFinal3,legPriceFinal4,m2m1,m2m2,m2m3,m2m4,totalPL,cumReturns,balance
0,22668.40,22618.45,22627.65,22646.50,2024-05-02 09:15:00+00:00,NaN,NaN,72.150000,Hold,2024-05-02 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,22674.35,22617.70,22646.25,22642.45,2024-05-02 09:30:00+00:00,NaN,NaN,66.983333,Hold,2024-05-02 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
2,22680.35,22634.25,22640.90,22669.50,2024-05-02 09:45:00+00:00,NaN,NaN,61.762500,Buy,2024-05-02 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
3,22677.45,22655.95,22669.00,22677.45,2024-05-02 10:00:00+00:00,NaN,NaN,53.710000,Hold,2024-05-02 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
4,22693.75,22669.60,22677.15,22684.15,2024-05-02 10:15:00+00:00,NaN,NaN,48.783333,Hold,2024-05-02 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0


In [5]:
# Read the options data
optionsData = pd.read_csv(f'data/{year}/NiftyOptionsData-{year}-15min.csv')

# optionsData.drop(['Unnamed: 0'], axis=1, inplace=True)
# Convert the 'Datetime' column to datetime format
optionsData['Datetime'] = pd.to_datetime(optionsData['Datetime'], format='%Y-%m-%d %H:%M:%S')
optionsData['ExpiryDate'] = pd.to_datetime(optionsData['ExpiryDate'], format='%Y-%m-%d %H:%M:%S%z')


# # keep only data where time is ending in 00, 15, 30, 45 mins
optionsData = optionsData[optionsData['Datetime'].dt.minute % 15 == 0]

optionsData = optionsData.sort_values(by='Datetime')
optionsData.reset_index(drop=True, inplace=True)


In [6]:
optionsData.head()

,Datetime,Open,High,Low,Close,Ticker,ExpiryDate,Strike,Instrument Type
0,2024-01-01 09:15:00,366.00,366.00,366.00,366.00,NIFTY24MAR21500PE.NFO,2024-03-28 00:00:00+00:00,21500.0,PE
1,2024-01-01 09:15:00,626.50,626.50,626.50,626.50,NIFTY24JAN21350CE.NFO,2024-01-25 00:00:00+00:00,21350.0,CE
2,2024-01-01 09:15:00,141.15,141.15,141.15,141.15,NIFTY24MAR20350PE.NFO,2024-03-28 00:00:00+00:00,20350.0,PE
3,2024-01-01 09:15:00,683.95,683.95,665.00,665.00,NIFTY24JAN21300CE.NFO,2024-01-25 00:00:00+00:00,21300.0,CE
4,2024-01-01 09:15:00,4.00,4.95,2.55,3.50,NIFTY2410420900PE.NFO,2024-01-04 00:00:00+00:00,20900.0,PE


In [7]:
# updated get close price fucntion
def get_close_price(strike_price, option_type, current_datetime, expiry_date):
    try:
        # First filter based on the first criteria
        
        # filtered_opd = optionsData.loc[(optionsData['Datetime'] == current_datetime) & (optionsData['Strike'] == strike_price) & (optionsData['ExpiryDate'] == expiry_date) & (optionsData['Instrument Type'] == option_type)]
            
        filtered_opd = optionsData[optionsData['Datetime'] == current_datetime]
        filtered_opd = filtered_opd[filtered_opd['Instrument Type'] == option_type]
        filtered_opd = filtered_opd[filtered_opd['Strike'] == strike_price]
        filtered_opd = filtered_opd[filtered_opd['ExpiryDate'] == expiry_date]
        
        

        # Now, apply the second criteria on the filtered DataFrame
        result = filtered_opd['Close']

        if result.empty:
            logging.warning(f"No close price found for {strike_price}, {option_type} at {current_datetime} for expiry {expiry_date}")
            return False
        else:
            return result.values[0]
    except Exception as e:
        logging.error(f"Error while fetching close price for {strike_price} at {current_datetime}")
        logging.error(e)



# get close price dict
def get_close_price_dict(atm_sp, wing_call, wing_put, current_datetime, expiry_date, i, df):
    try:
        # Define a list to store the futures
        futures = []

        # Create a ThreadPoolExecutor with a maximum of 4 threads
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            # Submit tasks to fetch close prices asynchronously
            futures.append(executor.submit(get_close_price, atm_sp, 'CE', current_datetime, expiry_date))
            futures.append(executor.submit(get_close_price, atm_sp, 'PE', current_datetime, expiry_date))
            futures.append(executor.submit(get_close_price, wing_call, 'CE', current_datetime, expiry_date))
            futures.append(executor.submit(get_close_price, wing_put, 'PE', current_datetime, expiry_date))

        # Retrieve the results
        results = [future.result() for future in futures]
        
        try:
            for j in range(len(results)):
                if results[j] == False:
                    previous_value = df.at[i - 1, f'legPriceOrignal{j + 1}']
                    if np.isnan(previous_value):
                        results[j] = 0
                    else:
                        results[j] = previous_value
        except Exception as e:
            print(f"Error in get_close_price_dict: {e}")

        # Map the results to the ticker names
        close_prices = {
            'atmSPCall': results[0],
            'atmSPPut': results[1],
            'wingCallPrice': results[2],
            'wingPutPrice': results[3]
        }
        
        
        return close_prices, df
    except Exception as e:
        logging.error(f"Error in get_close_price_dict: {e}")

In [8]:
# positions
positions = {'beginx': [-1, -1, 1, 1], 'buy': [0, -2, 2, 2], 'sell': [-2, 0, 2, 2], 'squareoff': [0, 0, 0, 0], 'hold': [0, 0, 0, 0], 'hard-squareoff': [0, 0, 0, 0]}

In [9]:
# Function to check Stop Loss
def check_stoploss(current_balance, capital_at_start_of_trade):
    # Calculate the difference in capital from the start to the current balance
    difference_in_capital = capital_at_start_of_trade - current_balance

    # Check if the difference is more than 3% of the initial capital
    return difference_in_capital > 0.03 * capital_at_start_of_trade

In [10]:
# m2m calculation
num_legs = 4

columns_to_reset = [ 'atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2',
    'legPriceOrignal3', 'legPriceOrignal4', 'balance', 'lpos1', 'lpos2', 'lpos3',
    'lpos4', 'legAfterPos1', 'legAfterPos2', 'legAfterPos3', 'legAfterPos4',
    'legAfterPosDiff1', 'legAfterPosDiff2', 'legAfterPosDiff3', 'legAfterPosDiff4',
    'legPriceFinal1', 'legPriceFinal2', 'legPriceFinal3', 'legPriceFinal4',
    'm2m1', 'm2m2', 'm2m3', 'm2m4', 'totalPL', 'cumReturns'
]

lpos_column_names = [f"lpos{i+1}" for i in range(num_legs)]


def calculate_m2m_new(index, start_day=False, caller='o'):
    position_to_take = positions[str(df.at[index, 'position'])]
    
    for i in range(0, num_legs):
        df.at[index, f"lpos{i+1}"] = position_to_take[i]
        # Calculating the leg after position
        df.at[index, f"legAfterPos{i+1}"] = df.at[index, f"legPriceOrignal{i+1}"] * df.at[index, f"lpos{i+1}"]
        

        # Calculating the difference between the leg after position and the previous leg after position
        difference_in_position = df.at[index, f"lpos{i+1}"] - df.at[index-1, f"lpos{i+1}"]
        df.at[index, f"legAfterPosDiff{i+1}"] = difference_in_position * df.at[index, f"legPriceOrignal{i+1}"]

        # Calculating Final Leg Price
        df.at[index, f"legPriceFinal{i+1}"] = df.at[index, f"legAfterPosDiff{i+1}"] + df.at[index-1, f"legAfterPos{i+1}"]

        if not start_day:
            # Calculating m2m
            df.at[index, f"m2m{i+1}"] = df.at[index, f"legPriceFinal{i+1}"] - df.at[index, f"legAfterPos{i+1}"]
        else:
            df.at[index, f"m2m{i+1}"] = 0

        
        # calculating the current data pnl which is the sum of all m2ms
        df.at[index, 'totalPL'] += df.at[index, f"m2m{i+1}"]

    # multiplying the totalPL by 50 for the lot size
    df.at[index, 'totalPL'] = df.at[index, 'totalPL'] * -1 * 50
    df.at[index, 'cumReturns'] = df.at[index-1, 'cumReturns'] + df.at[index, 'totalPL'] 
    # updating the balance
    df.at[index, 'balance'] = df.at[index-1, 'balance'] + df.at[index, 'totalPL']

In [11]:
squareoff_columns_reset = ['balance', 'lpos1', 'lpos2', 'lpos3',
    'lpos4', 'legAfterPos1', 'legAfterPos2', 'legAfterPos3', 'legAfterPos4',
    'legAfterPosDiff1', 'legAfterPosDiff2', 'legAfterPosDiff3', 'legAfterPosDiff4',
    'legPriceFinal1', 'legPriceFinal2', 'legPriceFinal3', 'legPriceFinal4',
    'm2m1', 'm2m2', 'm2m3', 'm2m4', 'totalPL', 'cumReturns']
 
# round to nearest 50
def round_to_nearest_50(number):
    return round(number / 50) * 50
    
for i in range(1, len(df)):
    df.at[df.index[i], 'atmSP'] = round_to_nearest_50(df.at[df.index[i], 'Close'])

In [12]:
# Function to determine if the current time is before the target time

def is_before_target_time(current_datetime, target_time="15:10:00+05:30"):
    target_time = datetime.strptime(target_time, "%H:%M:%S%z")
    # Compare the time parts
    return current_datetime.time() < target_time.time()


def get_daily_diff(current_datetime):
    day_of_week = current_datetime.weekday()
    # Define a dictionary to map each day to its corresponding difference
    day_diff_mapping = {
        "Monday": 400,
        "Tuesday": 300,
        "Wednesday": 200,
        "Thursday": 100,
        "Friday": 500,
        "Saturday": 0,
        "Sunday": 0
    }
    # Return the difference based on the day of the week
    return day_diff_mapping[datetime.strftime(current_datetime, '%A')]

In [13]:
# function to save data to csv
def save_data():
    df.to_csv(f'data/backtest/final/backtest{year}.csv')


In [14]:
logging.basicConfig(filename='logs.log', filemode='w', encoding='utf-8', level=logging.DEBUG, force=True)


In [15]:
position = "squareoff"

# set the initial capital and the current capital
capital = 100000

logging.info("*"*100)

highest_m2m = 0

first_run = True

for i, row in enumerate(df.itertuples()):
    # timer_start = tm.time()
    # Check if this a new day and time is 09:30:00+05:30

    current_datetime_str = str(row.Datetime)[:-6]
    current_datetime_str_short = current_datetime_str

    has_traded_today = False
    has_squareoff_today = False
    logging.info(f"Current Datetime: {current_datetime_str}")

    # check if the time is 09:30:00+05:30 or it is the first run
    if '09:45:00' in current_datetime_str or first_run:
        highest_m2m = 0
        # Calculate the ATM strike price
        atmSP = round_to_nearest_50(row.Close)
        wingPut = atmSP - get_daily_diff(row.Datetime)
        wingCall = atmSP + get_daily_diff(row.Datetime)
        
        # marking the postion as 0 updating tickers_dict
        close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, current_datetime_str_short, row.closest_expiry, i, df)

        df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4']] = [atmSP, wingCall, wingPut, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice']]
        if first_run:
            # Reset values to 0 for the specified columns at row i
            df.loc[i, columns_to_reset] = 0
            df.at[i, 'position'] = 'hold'
            position = 'hold'
            df.at[i, 'balance'] = capital
            first_run = False
        else:
            df.at[i, 'position'] = 'beginx'
            position = 'hold'
            calculate_m2m_new(i, start_day=True, caller='beginx 9:30')
            df.at[i, 'stoploss'] = highest_m2m - (100000*0.03)
        # sell atmSP call -> api call # sell atmSp put -> api call # buy wingPut -> api call # buy wingCall -> api call
    
    elif df.at[i-1, 'position'] == 'hard-squareoff' and row.Datetime.time() > time(9, 30):
        # if previous was hard-squareoff then hust continue and forward the values
        close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, current_datetime_str_short, row.closest_expiry, i, df)
        df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4']] = [atmSP, wingCall, wingPut, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice']]
        df.at[i, 'position'] = position = 'hard-squareoff'
        df.at[i, 'balance'] = df.at[i-1, 'balance']
        df.at[i, 'cumReturns'] = df.at[i-1, 'cumReturns']
        df.at[i, 'totalPL'] = 0

        
    
    # if time is 15:15:00+05:30 then squareoff all positions
    elif row.Datetime.time() == time(15, 15):
        # buy atmSp call -> api call # buy atmSp put -> api call # sell wingPut -> api call # sell wingCall -> api call
        capital = df.at[i-1, 'balance']
        df.at[i, 'balance'] = df.at[i-1, 'balance']
        df.at[i, 'position'] = position = 'squareoff'
        # store the values in the dataframe
        close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, current_datetime_str_short, row.closest_expiry, i, df)  
        df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4']] = [atmSP, wingCall, wingPut, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice']]         
        rolling_dict = calculate_m2m_new(i, caller='15:15')
    
    elif row.Datetime.time() == time(15, 30):
        # store the values in the dataframe
        df.loc[i, ['atmSP', 'wingCall', 'wingPut'] ] = [atmSP, wingCall, wingPut]
        df.at[df.index[i], 'position'] = 'hold'
        df.at[i, 'balance'] = df.at[i-1, 'balance']

        position = 'hold'
        
    elif row.Datetime.time() >= time(9, 15) and row.Datetime.time() < time(9, 45):
        # this is for time between 15:15:00+05:30 and 09:30:00+05:30
        # store the values in the dataframe

        df.loc[i, ['atmSP', 'wingCall', 'wingPut'] ] = [atmSP, wingCall, wingPut]
        df.at[df.index[i], 'position'] = 'hold'
        logging.debug(f"balance at 9:15: {df.at[i-1, 'balance']}")
        df.at[i, 'balance'] = df.at[i-1, 'balance']
        position = 'hold'
    
    elif df.at[i-1, 'position'] == 'squareoff' and row.Datetime.time() > time(9, 30):
        # if previous was squareoff then take position this time
        logging.info(f"Taking position at {current_datetime_str} by the elif block")
        atmSP = round_to_nearest_50(row.Close)
        wingPut = atmSP - get_daily_diff(row.Datetime)
        wingCall = atmSP + get_daily_diff(row.Datetime)
        close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, current_datetime_str_short, row.closest_expiry, i, df)  
        df.at[i, 'position'] = position = 'beginx'
        df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4']] = [atmSP, wingCall, wingPut, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice']]
        calculate_m2m_new(i, start_day=True, caller='beginx')

    
    else:
        close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, current_datetime_str_short, row.closest_expiry, i, df) 
        if position == 'hold':
            if row.signal == 'Buy':
                if row.rsi < 70 and row.ama < row.Close:
                    # buy atmSp call -> api call # Sell atmSp put -> api call # buy wingCall -> api call # buy wingPut -> api call
                    df.at[i, 'position'] = position = 'buy'
                    has_traded_today = True
            
            elif row.signal == 'Sell':
                if row.rsi > 30 and row.ama > row.Close:
                    # sell atmSp call -> api call # buy atmSp put -> api call # Buy wingPut -> api call # Buy wingCall -> api call
                    df.at[i, 'position'] = position = 'sell'
                    has_traded_today = True

        elif position == 'buy':
            if row.signal == 'Hold':
                if row.rsi > 70:
                    # buy atmSp put -> api call x 2 # sell wingPut -> api call x 2 # sell wingCall -> api call x 2
                    df.at[i, 'position'] = position = 'squareoff'
                    has_traded_today = True
            
            elif row.signal == 'Sell':          
                # buy atmSp put -> api call # sell wingPut -> api call # sell wingCall -> api call
                df.at[i, 'position'] = position =  'squareoff'
                has_traded_today = True

        elif position == 'sell':
            if row.signal == 'Hold':
                if row.rsi < 30:              
                    # buy atmSp call -> api call # sell wingPut -> api call # sell wingCall -> api call  
                    df.at[i, 'position'] = position = 'squareoff'
                    has_traded_today = True

                    
            elif row.signal == 'Buy':
                df.at[df.index[i], 'position'] = position = 'squareoff'
                has_traded_today = True
                

        df.at[i, 'position'] = position
        df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4']] = [atmSP, wingCall, wingPut, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice']]


        if not has_traded_today:
            df.at[i, 'position'] = df.at[i-1, 'position']

        calculate_m2m_new(i, caller='daily')
        
        if df.at[i, 'cumReturns'] > highest_m2m:
            highest_m2m = df.at[i, 'cumReturns']

        df.at[i, 'stoploss'] = stoploss_price = highest_m2m - (100000*0.03)
        # called everytime
        

        if df.at[i, 'cumReturns'] < stoploss_price:
            logging.info(f" Stoploss hit at {current_datetime_str}")
            # squareoff all positions
            df.loc[i, squareoff_columns_reset] = 0
            df.at[i, 'position'] = position = 'hard-squareoff'
            calculate_m2m_new(i, caller='stoploss')
            highest_m2m = df.at[i, 'cumReturns']
            # df.at[i, 'totalPL'] = -3300
            # df.at[i, 'balance'] = df.at[i-1, 'balance'] - 3300
            

        
    
    # timer_end = tm.time()
    # logging.info(f"Time taken for this iteration: {timer_end - timer_start} seconds")


    # df.at[df.index[i], 'position'] = position


save_data()





Error in get_close_price_dict: -1


/var/folders/fv/h8flhzz90ljgbvrrq5wxfv4w0000gn/T/ipykernel_12886/968494099.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4']] = [atmSP, wingCall, wingPut, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice']]


---
